## Projet de classification 

### Sujet : Bert_Dataset_2025

L'objectif est de Classifier la colonne "abstract" suivant les classes (6)

l'entrainement du model a été fait sur google Colab a partir de ce notebook 

	

In [ ]:
%nvidia-smi

Tue May 20 17:25:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%pip install torch transformers pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [17]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import numpy as np
import os


In [18]:
# Vérifier si le GPU est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [19]:
# charger les données

data = "/content/train 5.csv"
data = pd.read_csv(data)

data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [ ]:
# Classes cibles
labels = ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']

data[labels] = data[labels].fillna(0)
data['label'] = data[labels].idxmax(axis=1).map(lambda x: labels.index(x))

assert data['label'].min() >= 0 and data['label'].max() < len(labels), "Generated labels contain values outside the expected range [0, 5]"

# Dataset personnalisé
class AbstractDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [21]:
# Préparer les données
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
texts = data['ABSTRACT'].values
labels_data = data['label'].values

# Diviser les données
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels_data, test_size=0.2, random_state=42)

# Créer les datasets
train_dataset = AbstractDataset(train_texts, train_labels, tokenizer)
val_dataset = AbstractDataset(val_texts, val_labels, tokenizer)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Modèle
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
model.to(device)

# Optimiseur et perte
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Entraînement
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader)

    # Évaluation
def eval_model(model, data_loader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            # Renommer la variable locale pour éviter de masquer la liste globale des noms de classes
            true_batch_labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(true_batch_labels.cpu().numpy())

    
    all_possible_labels = list(range(len(labels))) 
    return accuracy_score(true_labels, predictions), classification_report(true_labels, predictions, target_names=labels, labels=all_possible_labels)

# Boucle d'entraînement
epochs = 3
for epoch in range(epochs):
    print(f'\nEpoch {epoch+1}/{epochs}')
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_acc, val_report = eval_model(model, val_loader, device)
    print(f'Train Loss: {train_loss:.4f}')
    print(f'Validation Accuracy: {val_acc:.4f}')
    print(val_report)


Epoch 1/3


Evaluating: 100%|██████████| 263/263 [02:11<00:00,  2.00it/s]


Train Loss: 0.3209
Validation Accuracy: 0.8060
                      precision    recall  f1-score   support

    Computer Science       0.80      0.88      0.84      1692
             Physics       0.94      0.86      0.90      1130
         Mathematics       0.80      0.82      0.81       898
          Statistics       0.42      0.31      0.36       343
Quantitative Biology       0.51      0.55      0.53        92
Quantitative Finance       0.76      0.78      0.77        40

            accuracy                           0.81      4195
           macro avg       0.71      0.70      0.70      4195
        weighted avg       0.80      0.81      0.80      4195


Epoch 2/3


Evaluating: 100%|██████████| 263/263 [02:11<00:00,  2.00it/s]


Train Loss: 0.2161
Validation Accuracy: 0.8052
                      precision    recall  f1-score   support

    Computer Science       0.78      0.92      0.84      1692
             Physics       0.92      0.90      0.91      1130
         Mathematics       0.89      0.72      0.80       898
          Statistics       0.39      0.30      0.34       343
Quantitative Biology       0.48      0.38      0.42        92
Quantitative Finance       0.79      0.47      0.59        40

            accuracy                           0.81      4195
           macro avg       0.71      0.62      0.65      4195
        weighted avg       0.80      0.81      0.80      4195


Epoch 3/3


Evaluating: 100%|██████████| 263/263 [02:11<00:00,  2.00it/s]

Train Loss: 0.1472
Validation Accuracy: 0.8005
                      precision    recall  f1-score   support

    Computer Science       0.79      0.87      0.83      1692
             Physics       0.93      0.87      0.90      1130
         Mathematics       0.80      0.82      0.81       898
          Statistics       0.39      0.28      0.33       343
Quantitative Biology       0.49      0.37      0.42        92
Quantitative Finance       0.74      0.62      0.68        40

            accuracy                           0.80      4195
           macro avg       0.69      0.64      0.66      4195
        weighted avg       0.79      0.80      0.79      4195



In [25]:
# sauvegarder le modele

model_save_path = 'abstract_classifier.pt'
torch.save(model.state_dict(), model_save_path)
from google.colab import files
files.download(model_save_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
# Sauvegarder le model sur le drive

model_save_path = '/content/drive/MyDrive/abstract_classifier.pt'
torch.save(model.state_dict(), model_save_path)

print(f"modèle save dans le drive à {model_save_path}")
from google.colab import drive
drive.mount('/content/drive')

modèle save dans le drive à /content/drive/MyDrive/abstract_classifier.pt
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
